In [ ]:
import pandas as pd
import os

# Block 1: File Loading with Error Handling
# ------------------------------
try:
    # Load calcium imaging data (neural activity signals)
    df_calcium = pd.read_csv("path_to_your_file.csv") # Change to your actual file path
    
    # Load speed data (movement measurements)
    df_speed = pd.read_csv("path_to_your_file.csv") # Change to your actual file path
except FileNotFoundError as e:
    print(f"Error: {e}. Please check file paths.")
    exit()

# Block 2: Define Time Range and Identify Columns
# ------------------------------
time_range = (0, 299.911)  # Time window in seconds for analysis

# Dynamically identify time columns (first column in each dataset)
time_col = df_calcium.columns[0]  # Time column for calcium data
speed_time_col = df_speed.columns[0]  # Time column for speed data

# Identify cell signal columns (all columns except first)
cell_columns = df_calcium.columns[1:]

# Block 3: Time Filtering with Optimized Query
# ------------------------------
# Filter datasets to specified time range using query() for better performance
df_calcium = df_calcium.query(f"{time_range[0]} <= {time_col} <= {time_range[1]}")
df_speed = df_speed.query(f"{time_range[0]} <= {speed_time_col} <= {time_range[1]}")

# Block 4: Time Binning with Efficient Vectorized Operations
# ------------------------------
# Create 1-second bins using floor division (vectorized operation)
df_calcium['Time_Bin'] = (df_calcium[time_col] // 1).astype(int)
df_speed['Time_Bin'] = (df_speed[speed_time_col] // 1).astype(int)

# Block 5: Calcium Data Aggregation with Named Aggregation
# ------------------------------
# Calculate mean and max for each cell per time bin
calcium_stats = df_calcium.groupby('Time_Bin').agg(
    **{f"{col}_mean": (col, 'mean') for col in cell_columns},
    **{f"{col}_max": (col, 'max') for col in cell_columns}
).reset_index()

# Block 6: Speed Data Aggregation
# ------------------------------
# Calculate mean and max speed per time bin
speed_stats = df_speed.groupby('Time_Bin').agg(
    mean_speed=('speed', 'mean'),
    max_speed=('speed', 'max')
).reset_index()

# Block 7: Data Merging with Validation
# ------------------------------
# Merge calcium and speed statistics on Time_Bin
result = pd.merge(calcium_stats, speed_stats, on='Time_Bin', how='inner')

# Validate merge success
if result.empty:
    print("Warning: No overlapping time bins between datasets!")
else:
    print(f"Successfully merged {len(result)} time bins")

# Block 8: Output with Path Validation
# ------------------------------
# Define output path
output_dir = "output_directory"  # Change to your desired output directory
csv_path = os.path.join(output_dir, "your-file-name.csv") # Change to your actual file name

# Ensure output directory exists
os.makedirs(output_dir, exist_ok=True)

# Save processed data
result.to_csv(csv_path, index=False)
print(f"Processed data saved to: {csv_path}")